## WideNet prototype

In [1]:
%run Setup.ipynb
%run ExtraFunctions.ipynb

[i] Loaded Parameters:
 40000 35 0.2 200 
 dataset/glove/glove.twitter.27B.200d.txt
[i] Importing Modules...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[+] Using Keras version 2.1.4
[i] Finished Importing Modules
[i] . Reading from csv file...Done!
[i] Found 30505 unique tokens.
[+] Shape of data tensor: (40000, 30)
[+] Shape of label tensor: (40000, 6)
[+] Number of entries in each category:
[+] Training:
 [ 7693.  7421.  4874. 10865.  1056.    91.]
[+] Validation:
 [1951. 1849. 1155. 2759.  267.   19.]
[i] Loading GloVe from: dataset/glove/glove.twitter.27B.200d.txt ...Done.
[+] Proceeding with Embedding Matrix...Completed!
Finished running setup.


In [2]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Feed the input vector into the embedding layer, then feed the resulting sequence into a bidirectional LSTM layer.

In [3]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm1 = Bidirectional(LSTM(8,return_sequences=True))(embedded_sequences)

In [4]:
inception = [] # append our layers in parallel

# kernel 7: best accuracy, some over fitting!
l_conv_7 = Conv1D(filters=24,kernel_size=7,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
# kernel 2: good accuracy
l_conv_2 = Conv1D(filters=24,kernel_size=2,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
# kernel 6: good accuracy
l_conv_6 = Conv1D(filters=24,kernel_size=6,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
# kernel 3: good accuracy, no over fitting
l_conv_3 = Conv1D(filters=24,kernel_size=3,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)

inception.append(l_conv_7)
inception.append(l_conv_2)
inception.append(l_conv_6)
inception.append(l_conv_3)

# poorer performing layers
l_conv_4 = Conv1D(filters=24,kernel_size=4,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
l_drop_4 = Dropout(0.6)(l_conv_4)
l_conv_5 = Conv1D(filters=24,kernel_size=5,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
l_drop_5 = Dropout(0.6)(l_conv_5)
l_conv_8 = Conv1D(filters=24,kernel_size=8,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
l_drop_8 = Dropout(0.6)(l_conv_8)

inception.append(l_drop_4)
inception.append(l_drop_5)
inception.append(l_drop_8)

In [5]:
# last parallel layer in inception: max-pooling layer
l_pool_i1 = MaxPooling1D(3)(l_lstm1)
l_conv_1 = Conv1D(filters=24,kernel_size=1,
                    activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_pool_i1)
inception.append(l_conv_1)

In [6]:
l_merge = Concatenate(axis=1)(inception)
l_pool = MaxPooling1D(4)(l_merge)
l_drop = Dropout(0.5)(l_pool)
l_flat = Flatten()(l_drop)
l_dense = Dense(12, activation='relu')(l_flat)
preds = Dense(6, activation='softmax')(l_dense)

In [7]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
lr_metric = get_lr_metric(adadelta)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [8]:
!rm -r logs

In [9]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoint.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [10]:
model.summary()
model.save('WideNet.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 200)      6101200     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 16)       13376       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 27, 24)       1560        bidirectional_1[0][0]            
__________________________________________________________________________________________________
conv1d_6 (

In [11]:
print("Training Progress:")
model_log = model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=50,
          callbacks=[tensorboard, lr_schedule])

Training Progress:
Train on 32000 samples, validate on 8000 samples
Epoch 1/200
32000/32000 [==============================] - 21s 664us/step - loss: 1.5478 - acc: 0.3359 - val_loss: 1.4605 - val_acc: 0.3449
Epoch 2/200
32000/32000 [==============================] - 21s 662us/step - loss: 1.4669 - acc: 0.3395 - val_loss: 1.4577 - val_acc: 0.3449
Epoch 3/200
32000/32000 [==============================] - 21s 658us/step - loss: 1.4670 - acc: 0.3395 - val_loss: 1.4578 - val_acc: 0.3449
Epoch 4/200
32000/32000 [==============================] - 21s 670us/step - loss: 1.4669 - acc: 0.3395 - val_loss: 1.4585 - val_acc: 0.3449
Epoch 5/200
32000/32000 [==============================] - 21s 661us/step - loss: 1.4661 - acc: 0.3395 - val_loss: 1.4590 - val_acc: 0.3449
Epoch 6/200
32000/32000 [==============================] - 21s 656us/step - loss: 1.4664 - acc: 0.3395 - val_loss: 1.4570 - val_acc: 0.3449
Epoch 7/200
32000/32000 [==============================] - 21s 654us/step - loss: 1.4649 - a

32000/32000 [==============================] - 21s 647us/step - loss: 1.1172 - acc: 0.5778 - val_loss: 1.3210 - val_acc: 0.4906
Epoch 59/200
32000/32000 [==============================] - 21s 649us/step - loss: 1.1173 - acc: 0.5786 - val_loss: 1.3353 - val_acc: 0.4834
Epoch 60/200
32000/32000 [==============================] - 21s 648us/step - loss: 1.1130 - acc: 0.5787 - val_loss: 1.3287 - val_acc: 0.4785
Epoch 61/200
32000/32000 [==============================] - 21s 654us/step - loss: 1.1121 - acc: 0.5819 - val_loss: 1.3473 - val_acc: 0.4854
Epoch 62/200
32000/32000 [==============================] - 21s 649us/step - loss: 1.1108 - acc: 0.5805 - val_loss: 1.3605 - val_acc: 0.4810
Epoch 63/200
32000/32000 [==============================] - 21s 650us/step - loss: 1.1074 - acc: 0.5833 - val_loss: 1.3630 - val_acc: 0.4809
Epoch 64/200
32000/32000 [==============================] - 21s 649us/step - loss: 1.1064 - acc: 0.5831 - val_loss: 1.3450 - val_acc: 0.4839
Epoch 65/200
32000/32000 [

32000/32000 [==============================] - 19s 609us/step - loss: 1.0232 - acc: 0.6219 - val_loss: 1.4328 - val_acc: 0.4614
Epoch 117/200
32000/32000 [==============================] - 20s 613us/step - loss: 1.0209 - acc: 0.6228 - val_loss: 1.4560 - val_acc: 0.4746
Epoch 118/200
32000/32000 [==============================] - 20s 614us/step - loss: 1.0193 - acc: 0.6233 - val_loss: 1.4218 - val_acc: 0.4755
Epoch 119/200
32000/32000 [==============================] - 20s 613us/step - loss: 1.0170 - acc: 0.6217 - val_loss: 1.4511 - val_acc: 0.4701
Epoch 120/200
32000/32000 [==============================] - 20s 615us/step - loss: 1.0198 - acc: 0.6235 - val_loss: 1.4591 - val_acc: 0.4706
Epoch 121/200
32000/32000 [==============================] - 19s 609us/step - loss: 1.0189 - acc: 0.6248 - val_loss: 1.4312 - val_acc: 0.4695
Epoch 122/200
32000/32000 [==============================] - 20s 611us/step - loss: 1.0162 - acc: 0.6256 - val_loss: 1.4487 - val_acc: 0.4694
Epoch 123/200
32000/

Epoch 174/200
32000/32000 [==============================] - 20s 610us/step - loss: 0.9794 - acc: 0.6423 - val_loss: 1.5132 - val_acc: 0.4665
Epoch 175/200
32000/32000 [==============================] - 20s 612us/step - loss: 0.9765 - acc: 0.6448 - val_loss: 1.5548 - val_acc: 0.4681
Epoch 176/200
32000/32000 [==============================] - 20s 611us/step - loss: 0.9743 - acc: 0.6442 - val_loss: 1.5273 - val_acc: 0.4690
Epoch 177/200
32000/32000 [==============================] - 20s 610us/step - loss: 0.9781 - acc: 0.6420 - val_loss: 1.5480 - val_acc: 0.4569
Epoch 178/200
32000/32000 [==============================] - 20s 611us/step - loss: 0.9751 - acc: 0.6440 - val_loss: 1.5051 - val_acc: 0.4716
Epoch 179/200
32000/32000 [==============================] - 20s 613us/step - loss: 0.9742 - acc: 0.6454 - val_loss: 1.5166 - val_acc: 0.4673
Epoch 180/200
32000/32000 [==============================] - 19s 609us/step - loss: 0.9766 - acc: 0.6420 - val_loss: 1.4936 - val_acc: 0.4646
Epoch 

In [12]:
import pandas as pd
model.save('WideNet.h5')
pd.DataFrame(model_log.history).to_csv("history-inception.csv")